<a href="https://colab.research.google.com/github/AdityaMahaddalkar/pyspark-prototype/blob/master/PySpark_Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!wget 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv' -O 'titanic.csv'

--2023-03-02 14:57:58--  https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60302 (59K) [text/plain]
Saving to: ‘titanic.csv’

titanic.csv         100%[===================>]  58.89K  --.-KB/s    in 0.007s  

2023-03-02 14:57:58 (7.69 MB/s) - ‘titanic.csv’ saved [60302/60302]



In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('TitanicAnalyzer').getOrCreate()

df = spark.read.csv('titanic.csv', header=True, inferSchema=True)

df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

### Drop null rows in data

In [4]:
df = df.dropna(how="any")

df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+-----------+--------+-----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|     Ticket|    Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+-----------+--------+-----------+--------+
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|   PC 17599| 71.2833|        C85|       C|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|     113803|    53.1|       C123|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|54.0|    0|    0|      17463| 51.8625|        E46|       S|
|         11|       1|     3|Sandstrom, Miss. ...|female| 4.0|    1|    1|    PP 9549|    16.7|         G6|       S|
|         12|       1|     1|Bonnell, Miss. El...|female|58.0|    0|    0|     113783|   26.55|       C103|       S|
|         22|       1|     2|Beesley, Mr. Lawr...|  male|34.0|  

### Give average Age grouped by Sex

In [5]:
from pyspark.sql.functions import avg

avg_age_by_sex = df.groupBy("Sex").agg(avg("Age").alias("Average Age"))

avg_age_by_sex.show()

+------+-----------------+
|   Sex|      Average Age|
+------+-----------------+
|female|32.67613636363637|
|  male|38.45178947368421|
+------+-----------------+



### Give average Fare grouped by PClass

In [6]:
avg_fare_by_pclass = df.groupBy("PClass").agg(avg("Fare").alias("Average Fare"))

avg_fare_by_pclass.show()

+------+------------------+
|PClass|      Average Fare|
+------+------------------+
|     1| 88.68322848101265|
|     3|           11.0275|
|     2|18.444446666666668|
+------+------------------+



### Find percentage of males and females who survived

In [7]:
survived_df = df.filter(df.Survived == 1)

counts_by_sex = survived_df.groupBy("Sex").count()

total_count = survived_df.count()

percentage_male = counts_by_sex.filter(counts_by_sex.Sex == "male").selectExpr("count / {} * 100 as percentage".format(total_count)).collect()[0]["percentage"]
percentage_female = counts_by_sex.filter(counts_by_sex.Sex == "female").selectExpr("count / {} * 100 as percentage".format(total_count)).collect()[0]["percentage"]

print("Percentage of males who survived: {:.2f}%".format(percentage_male))
print("Percentage of females who survived: {:.2f}%".format(percentage_female))

Percentage of males who survived: 33.33%
Percentage of females who survived: 66.67%


### Print all distinct values of column Embarked

In [8]:
distinct_values = df.select("Embarked").distinct().collect()

print("Distinct values of 'Embarked' column:")
for row in distinct_values:
    print(row[0])

Distinct values of 'Embarked' column:
Q
C
S


### Split 'Name' column into First and Last name

In [9]:
from pyspark.sql.functions import split
from pyspark.sql.functions import trim

df = df.withColumn('Last Name', trim(split(df['Name'], ',').getItem(0)))
df = df.withColumn('First Part of Name', trim(split(df['Name'], ',').getItem(1)))

df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+-----------+--------+-----------+--------+-----------+--------------------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|     Ticket|    Fare|      Cabin|Embarked|  Last Name|  First Part of Name|
+-----------+--------+------+--------------------+------+----+-----+-----+-----------+--------+-----------+--------+-----------+--------------------+
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|   PC 17599| 71.2833|        C85|       C|    Cumings|Mrs. John Bradley...|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|     113803|    53.1|       C123|       S|   Futrelle|Mrs. Jacques Heat...|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|54.0|    0|    0|      17463| 51.8625|        E46|       S|   McCarthy|       Mr. Timothy J|
|         11|       1|     3|Sandstrom, Miss. ...|female| 4.0|    1|    1|    PP 9549|    16.7|     

### Split First Name into "Initials First Name Middle Name"

In [10]:
df = df.withColumn('Initials', trim(split(df['First Part of Name'], ' ').getItem(0)))
df = df.withColumn('First Name', trim(split(df['First Part of Name'], ' ').getItem(1)))
df = df.withColumn('Middle Name', trim(split(df['First Part of Name'], ' ').getItem(2)))

df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+-----------+--------+-----------+--------+-----------+--------------------+--------+----------+-----------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|     Ticket|    Fare|      Cabin|Embarked|  Last Name|  First Part of Name|Initials|First Name|Middle Name|
+-----------+--------+------+--------------------+------+----+-----+-----+-----------+--------+-----------+--------+-----------+--------------------+--------+----------+-----------+
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|   PC 17599| 71.2833|        C85|       C|    Cumings|Mrs. John Bradley...|    Mrs.|      John|    Bradley|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|     113803|    53.1|       C123|       S|   Futrelle|Mrs. Jacques Heat...|    Mrs.|   Jacques|      Heath|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|54.0|    0|    0|      17463| 51.

In [11]:
df_grouped = df.groupBy('Last Name').count().orderBy('count', ascending=False)

df_grouped.show()

+--------------------+-----+
|           Last Name|count|
+--------------------+-----+
|             Fortune|    4|
|              Carter|    4|
|              Thayer|    3|
|             Allison|    3|
|              Newell|    3|
|              Graham|    3|
|             Taussig|    3|
|              Baxter|    2|
|               Pears|    2|
|             Minahan|    2|
|                Hays|    2|
|             Andrews|    2|
|              Harris|    2|
|        "Duff Gordon|    2|
|                Dick|    2|
|               White|    2|
|Penasco y Castellana|    2|
|            Chambers|    2|
|                Moor|    2|
|                Hoyt|    2|
+--------------------+-----+
only showing top 20 rows

